In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.express as px
from utils.metrics import construct_dataframe_faces
from utils.imgs import get_bbox_size
import plotly.graph_objects as go

In [14]:
subsets = [
    'base_one_person_dreamshaper',
    'base_two_people_dreamshaper',
    'ip_one_person',
    'ip_one_person_wp5',
    'ip_one_person_wp25',
    'two_people_faceid_dreamshaper',
]
subset_name = subsets[0]

In [15]:
df_raw = construct_dataframe_faces(subset_name)

In [16]:
df = df_raw.drop(columns=['kps'])

In [17]:
df['bbox_size'] = df['bbox'].apply(lambda x: get_bbox_size(x))

In [18]:
df.head()

,subset,image_id,bbox,det_score,face_quality,bbox_size
0,base_one_person_dreamshaper,0,"[478, 189, 636, 383]",0.892645,0.677580,30652
1,base_one_person_dreamshaper,1,"[466, 138, 599, 313]",0.809491,0.440618,23275
2,base_one_person_dreamshaper,1,"[159, 397, 182, 431]",0.761878,0.088538,782
3,base_one_person_dreamshaper,2,"[412, 202, 536, 371]",0.814545,0.792386,20956
4,base_one_person_dreamshaper,2,"[147, 333, 159, 349]",0.539451,0.298617,192


In [19]:
df = df.sort_values('bbox_size', ascending=False).groupby('image_id').first().reset_index()
y_mean = df['face_quality'].mean()
y_std = df['face_quality'].std()

In [21]:
fig = px.histogram(df, x='face_quality', title='Face quality when using just a prompt on single person generation')
fig.update_layout(
        width=1000,  # Set a custom width in pixels
    height=600,
    autosize=True, 
    plot_bgcolor='white',
    paper_bgcolor='white', 
    font=dict(color='black'),  # Set font color to black for better contrast
)
# Add mean line
fig.add_vline(x=y_mean, line_dash="dash", line_color="red", annotation_text=f"Mean: {y_mean:.2f}", 
              annotation_position="top right")

# Add +1 std line
fig.add_vline(x=y_mean - y_std, line_dash="dot", line_color="green", annotation_text=f"-1 std: {y_mean - y_std:.2f}", 
              annotation_position="top right")

fig.add_vline(x=y_mean + y_std, line_dash="dot", line_color="green", annotation_text=f"+1 std: {y_mean + y_std:.2f}", 
              annotation_position="top right")


In [20]:
250 * 250

62500